In [ ]:
from sklearn.metrics import cohen_kappa_score
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import *
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import resample
from sklearn.metrics import confusion_matrix
import io
import os
import struct
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import pathlib
from scipy import signal
import matplotlib.pyplot as plt
import scipy.signal as signal
from scipy import fftpack
import math
from matplotlib import patches
import matplotlib.ticker as mtick
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import average_precision_score

In [ ]:
def read_pickle(file_dir):   
    pickle_L=[]   
    pickle_name=[]
    for dirpath, dirnames, filenames in os.walk(file_dir):  
        for file in filenames :  
            if os.path.splitext(file)[1] == '.pickle':  
                pickle_L.append(os.path.join(dirpath, file))  
                pickle_name.append(os.path.join(file))  
            
    #return L 
    return  pickle_name
    #return name

In [ ]:
def pickleLoader(pklFile):
    try:
        while True:
            yield pickle.load(pklFile)
    except EOFError:
        pass

In [ ]:
from itertools import groupby
from operator import itemgetter

def time(nums):
    start=[]
    end=[]
    arr=[]
    for k, g in groupby(enumerate(nums), lambda ix : ix[0] - ix[1]):
        arr.append(list(map(itemgetter(1), g)))
    for j in range (0,len(arr)):
        if (len(arr[j])>=2):
            start.append(arr[j][0])
            end.append(arr[j][-1])
    return start, end

In [ ]:
def findSmallest(arr):
    smallest = arr[0]#将第一个元素的值作为最小值赋给smallest    
    smallest_index = 0#将第一个值的索引作为最小值的索引赋给smallest_index    
    for i in range(1, len(arr)):        
        if arr[i] < smallest:#对列表arr中的元素进行一一对比            
            smallest = arr[i]            
            smallest_index = i    
    return smallest_index
def selectionSort(arr):    
    newArr = []    
    for i in range(len(arr)):        
        smallest = findSmallest(arr)#一共要调用5次findSmallest        
        newArr.append(arr.pop(smallest))#每一次都把findSmallest里面的最小值删除并存放在新的数组newArr中    
    return newArr

# Load model 

In [ ]:
import joblib
#load model
filename='JNE_XG_W478.sav'

rf=joblib.load(filename)

In [ ]:
threshold=0.5

# Model

In [ ]:
tra_path='Epi-AI_processed_files/Test/Model_I/'
#tra_path='Epi-AI_processed_files/Test/Model_II/'
#tra_path='Epi-AI_processed_files/Test/Model_III/'
#tra_path='Epi-AI_processed_files/Test/Model_IV/'

In [ ]:
train_file=read_pickle(tra_path)

In [ ]:
sepcificity=[]
sensitivity=[]
accuracy=[]
ROC=[]
file_name=[]


for i in range (0, len(train_file)):
    with open(tra_path+train_file[i], 'rb') as file:
        #print(train_file[i])
        file_name.append(train_file[i])
        for event in pickleLoader(file):
            #print(event)
            data_test=event[['target','mean_EEG2', 'EEG2_mobility', 'TKEO_EEG2', 'EEG2_P_delta','EEG2_P_theta',
                                'EEG2_P_alpha','EEG2_P_beta','EEG2_P_gamma','EEG2_P_total','EEG2_rel_delta','EEG2_rel_theta',
                                'EEG2_rel_alpha','EEG2_rel_beta','EEG2_rel_gamma','EEG2_pfd','EEG2_skew',
                                'EEG2_kurtosis','EEG2_var','EEG2_envelope']]
            
            data_test.columns=['target','mean', 'mobility','TKEO', 'absolute delta power','absolute theta power',
               'absolute alpha power','absolute beta power','absolute gamma power','absolute total power',
               'relative delta power','relative theta power','relative alpha power','relative beta power',
               'relative gamma power','fractal dimension','skewness',
                'kurtosis','variance','signal envelope']
     
            data_test=data_test.reset_index(drop=True)
            
            train_X = data_test.drop('target', axis=1)
            train_y=data_test.target

            manual=train_y.where(train_y==1)
            manual=manual.dropna()
            manunum=len(time(manual.index)[0])

            predicted_proba = rf.predict_proba(train_X)
            train_pred= (predicted_proba [:,1] >= threshold).astype('int')
            
            train_pred=pd.DataFrame(train_pred)
  
            pred=train_pred
            auto=pred.where(pred==1)
            auto=auto.dropna()
            val=time(auto.index)
            start_ti=[-1]
            end_ti=[-1]
        
            
            for jjj in range(0, len(val[0])):
                start_ti.append(val[0][jjj])
                end_ti.append(val[1][jjj])
           
            start_ti=pd.DataFrame(start_ti)
            end_ti=pd.DataFrame(end_ti)

            frame=[start_ti,end_ti]
            performance=pd.concat(frame,axis=1)
            performance.columns=['start_time','end_time']
            
            spindle_start=start_ti
            spindle_end=end_ti
    
            spindle_start=pd.DataFrame(spindle_start)
            spindle_end=pd.DataFrame(spindle_end)
            spindle_dur=spindle_end-spindle_start
            
            frame0=[spindle_start,spindle_end,spindle_dur]
            spindle_time=pd.concat(frame0,axis=1)
            spindle_time.columns=['spindle_start','spindle_end','spindle_dur']
            spindle_time= spindle_time.drop_duplicates(subset=['spindle_end'], keep='first')
            spindle_time = spindle_time[spindle_time['spindle_dur'] >=2]
            #print(len(spindle_time))
            #print(len(spindle_time))
            spindle_time=spindle_time.reset_index(drop=True)
            
            #ppp='D:/desk/EEG/UCC_infant/remove_abmornal/auto_label/'
            #pathlib.Path(ppp).mkdir(parents=True, exist_ok=True) 
            
            #spindle_time.to_csv(ppp+'time_'+train_file[i]+'.csv')
            
            start_com=pd.DataFrame(start_ti)
            end_com=pd.DataFrame(end_ti)
            start_com=(start_com).astype(int)
            end_com=(end_com).astype(int)+1
            start_com=pd.DataFrame(start_com)
            end_com=pd.DataFrame(end_com)
            dur_com=end_com-start_com
            
            frame=[start_com,end_com,dur_com]
            time_ta=pd.concat(frame,axis=1)
            time_ta.columns=['start_com','end_com','dur_com']
            time_table= time_ta.drop_duplicates(subset=['end_com'], keep='first')
            #print(len(time_table))
            time_table = time_table[time_table['dur_com'] >=2]
            #print(len(time_table))
            time_table=time_table.reset_index(drop=True)
            
            autonum=len(time_table)
            
            spindle_time['Start_signal']=(spindle_time['spindle_start'])
            spindle_time['End_signal']=(spindle_time['spindle_end'])
            spindle_time['Duration']=spindle_time['End_signal']-spindle_time['Start_signal']

            data_test['tkeo_label']=0
            for i in range(0,len(spindle_time)):
                data_test['tkeo_label'][spindle_time['Start_signal'].values[i]-3:spindle_time['End_signal'].values[i]+3]=1
                
           
            ind=[-5]
            index=[-6]

            for iiii in range(0, len(train_y)):
                if ( data_test['tkeo_label'][iiii]==1 & train_y[iiii]==1):
                    ind.append(iiii)
            
            #print(ind[:200])
            index.append(ind[0])    
            for jjjj in range (1, len(ind)):
                if ind[jjjj]-ind[jjjj-1]!=1:
                    index.append(ind[jjjj])
            
            tn, fp, fn, tp = confusion_matrix(train_y, data_test['tkeo_label'], labels=[0,1]).ravel()
            spec = tn / (tn+fp)
            sens=tp/(tp+fn)
            falsepr=fp/(fp+tn)
            falsedr=fp/(fp+tp)
           
            sepcificity.append(spec)
            sensitivity.append(sens)
            accuracy.append(metrics.accuracy_score(train_y,data_test['tkeo_label']))
            

            data_test['tkeo_label'][0]=1
            train_y[0]=1
            ROC.append(roc_auc_score(train_y,data_test['tkeo_label']))

In [ ]:
file_name=pd.DataFrame(file_name)
sepcificity=pd.DataFrame(sepcificity)
sensitivity=pd.DataFrame(sensitivity)
accuracy=pd.DataFrame(accuracy)
ROC=pd.DataFrame(ROC)

In [ ]:
frame=[file_name,sepcificity,sensitivity,accuracy,ROC]
performance=pd.concat(frame,axis=1)
performance.columns=['file_name','sepcificity','sensitivity','accuracy','ROC']

In [ ]:
performance.to_csv('Epi-AI_Model_I_Performance.csv')
#performance.to_csv('Epi-AI_Model_II_Performance.csv')
#performance.to_csv('Epi-AI_Model_III_Performance.csv')
#performance.to_csv('Epi-AI_Model_IV_Performance.csv')